In [ ]:
import torch
import csv
import gzip
import nltk
from torch.utils.data import TensorDataset

In [ ]:
# Dictionary structure 
import gzip
import torch.nn as nn
from torch.autograd import Variable
from dpp_nets.utils.io import embd_iterator, make_embd
from collections import namedtuple
from nltk import word_tokenize
import torch
import numpy as np
from dpp_nets.my_torch.utilities import pad_tensor

id_to_content = {}
errors = []
#embd_path = '/Users/Max/data/askubuntu/vectors_stackexchange.txt.gz'
embd_path = '/Users/Max/data/askubuntu/vectors_pruned.200.txt.gz'
embd_layer, word_to_ix  = make_embd(embd_path)
question = namedtuple('question', 'id title body title_ix body_ix')
path = '/Users/Max/data/askubuntu/text_tokenized.txt.gz'

In [ ]:
val_path = '/Users/Max/data/askubuntu/dev.txt'
count = 0
import torch

with open(val_path) as f:
    data_tensor = []
    target_tensor = []
    
    for line in f:
        q_id, pos, candidates, _ = line.split("\t")
        q_id = int(q_id)
        pos = [int(id) for id in pos.split()]
        candidates = [int(id) for id in candidates.split()]
        
        # Check
        if len(pos) == len(candidates):
            continue
        
        # Create target
        target = [1 if i in pos else 0 for i in candidates]
        target = [-1] + target
        target = torch.ByteTensor(target)
        

In [ ]:
        # Create x Tensor 
        all_ids = [q_id] + candidates 
        all_ids = [id_to_content[id] for id in all_ids]
        all_ids = [pad_tensor(torch.LongTensor(np.array(id.title_ix)),0,0,max_title_size) for id in all_ids]
        all_ids = torch.stack(all_ids)
        data_tensor.append(all_ids)
        target_tensor.append(target)
        
    data_tensor = torch.stack(data_tensor)
    target_tensor = torch.stack(target_tensor)

In [ ]:
val_ds.target_tensor.size()

In [ ]:
val_ds.data_tensor.size()

In [ ]:
import torch.nn as nn

In [ ]:
nn.CosineEmbeddingLoss

In [ ]:
embd_layer, word_to_ix  = make_embd(embd_path)

In [ ]:
id_to_content, word_to_ix, max_title_size, max_body_size = create_id_to_content(data_base_path, word_to_ix)

In [ ]:
embd = update_embd(embd_layer, word_to_ix)

In [ ]:
title_ds, _ = create_train_set(train_path, id_to_content, max_title_size, max_body_size)

In [ ]:
titles = []
bodies = []
targets = []

with open(train_path) as f:
    for line in f:
        q_id, pos, neg = line.split("\t")
        q_id = int(q_id)
        q_content = id_to_content[q_id]
        q_title = pad_tensor(torch.LongTensor(np.array(q_content.title_ix)),0,0, max_title_size)
        q_body = pad_tensor(torch.LongTensor(np.array(q_content.body_ix)), 0,0, max_body_size)
        
        pos = [id_to_content[int(id)] for id in pos.split()]
        neg = [id_to_content[int(id)] for id in neg.split()]
        
        targets = [for q in ]
        targets.extend([1] * len(pos))
        targets.extend([-1] * len(neg))

        pos_pairs_title = [torch.stack([q_title, pad_tensor(torch.LongTensor(np.array(q.title_ix)),0,0,max_title_size)]) 
                           for q in pos]
        neg_pairs_title = [torch.stack([q_title, pad_tensor(torch.LongTensor(np.array(q.title_ix)),0,0,max_title_size)])
                            for q in neg]

        titles.extend(pos_pairs_title)
        titles.extend(neg_pairs_title)        

        pos_pairs_body = [torch.stack([q_body, pad_tensor(torch.LongTensor(np.array(q.body_ix)),0,0,max_body_size)]) 
                           for q in pos]
        neg_pairs_body = [torch.stack([q_body, pad_tensor(torch.LongTensor(np.array(q.body_ix)),0,0,max_body_size)]) 
                           for q in neg]

        bodies.extend(pos_pairs_body)
        bodies.extend(neg_pairs_body)

In [1]:
embd_path = '/Users/Max/data/askubuntu/vectors_pruned.200.txt.gz'
data_base_path = '/Users/Max/data/askubuntu/text_tokenized.txt.gz'
train_path = '/Users/Max/data/askubuntu/train_random.txt'
val_path = '/Users/Max/data/askubuntu/dev.txt'

from dpp_nets.utils.ubuntu_io import create_ubuntu 

In [2]:
embd, train_ds, val_ds = create_ubuntu(embd_path, data_base_path, train_path, val_path)

In [43]:
train_set, val_set = train_ds, val_ds
import argparse
import os
import shutil

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.utils.data.dataloader import DataLoader

from dpp_nets.utils.ubuntu_io import compute_ap, create_ubuntu
from dpp_nets.layers.layers import DeepSetBaseline

import numpy as np
batch_size = 10
lr = 0.001

embd.requires_grad = False

torch.manual_seed(0)
train_loader = DataLoader(train_set, batch_size, shuffle=True)
val_loader = DataLoader(val_set, 1)
print("loader defined")

### Build model
# Network parameters
embd_dim = embd.weight.size(1)
hidden_dim = 500
enc_dim = 200
target_dim = 200

# Model
torch.manual_seed(0)
net = DeepSetBaseline(embd_dim, hidden_dim, enc_dim, target_dim)
activation = nn.Sigmoid()
model = nn.Sequential(embd, net, activation)
print("created model")

### Set-up training
criterion = nn.CosineEmbeddingLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
print("set up optimizer")

### Loop
torch.manual_seed(0)
print("started loop")


loader defined
created model
set up optimizer
started loop


In [23]:
remote = 0
ckp_path_local = '/can_be_deleted'
global lr

def train(loader, model, criterion, optimizer):

    model.train()

    for t, (qs, target) in enumerate(loader):
        
        q1 = Variable(qs[:,0,:])
        q2 = Variable(qs[:,1,:])
        target = Variable(target)

        pred1 = model(q1)
        pred2 = model(q2)
        loss = criterion(pred1, pred2, target)

        optimizer.zero_grad()
        loss.backward() # I'm currently not using the Hinge loss. 
        optimizer.step()

        print("trained one batch")

def validate(loader, model, criterion):

    model.eval()

    MAP = 0.0

    for t, (qs, target) in enumerate(loader, 1):

        q0 = Variable(qs[:,0,:])
        pred0 = model(q0)

        scores = []
        losses = []

        for i in range(1, qs.size(1)):

            q = Variable(qs[:,i,:], volatile=True)
            pred = model(q)
            target = Variable(target[i], volatile=True)
            print(pred.size(), pred0.size())
            print(pred0.pow(2).sum().sqrt() * pred.pow(2).sum().sqrt())
            print((pred * pred0).sum())
            cos_dis = 1 - ((pred * pred0).sum(1) / (pred0.pow(2).sum(1).sqrt() * pred.pow(2).sum(1).sqrt())) 
            scores.append(cos_dis.data[0])
            loss = criterion(pred0, pred, target)
            losses.append(loss.data[0])

        scores = np.array(scores)
        target = target.numpy()
        ap = compute_ap(scores, append)
        delta = ap - MAP
        MAP += (delta / t)

        average_loss = losses.sum() / len(loss)

        print("validated one batch")

    return average_loss, MAP

def log(epoch, loss, MAP):
    string = str.join(" | ", ['Epoch: %d' % (epoch), 'Validation Loss: %.5f' % (loss), 'Validation MAP: %.5f' % (MAP)])

    if remote:
        destination = os.path.join(ckp_path_remote, aspect + str(lr) + 'ubuntu_baseline_log.txt')
    else:
        destination = os.path.join(ckp_path_local, aspect + str(lr) + 'ubuntu_baseline_log.txt')

    with open(destination, 'a') as log:
        log.write(string + '\n')

def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR multiplied by factor 0.1 for every 20 epochs"""
    lr = .2
    lr = lr * (0.1 ** (epoch // 25))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

def save_checkpoint(state, is_best, filename='ubuntu_baseline_checkpoint.pth.tar'):
    """
    State is a dictionary that cotains valuable information to be saved.
    """
    if remote:
        destination = os.path.join(ckp_path_remote, aspect + str(lr) + filename)
    else:
        destination = os.path.join(ckp_path_local, aspect + str(lr) + filename)
    
    torch.save(state, destination)
    if is_best:
        if remote:
            best_destination = os.path.join(ckp_path_remote, aspect + str(lr) + 'ubuntu_baseline_model_best.pth.tar')
        else:
            best_destination = os.path.join(ckp_path_local, aspect + str(lr) + 'ubuntu_baseline_model_best.pth.tar')
        
        shutil.copyfile(destination, best_destination)


In [24]:
epochs = 2
for epoch in range(epochs):

    adjust_learning_rate(optimizer, epoch)

    #train(train_loader, model, criterion, optimizer)        
    loss, MAP = validate(val_loader, model, criterion)

    log(epoch, loss, MAP)
    print("logged")

    is_best = loss < lowest_loss
    lowest_loss = min(loss, lowest_loss)    
    save = {'epoch:': epoch + 1, 
            'model': 'Deep Set Baseline',
            'state_dict': model.state_dict(),
            'lowest_loss': lowest_loss,
            'optimizer': optimizer.state_dict()} 

    save_checkpoint(save, is_best)
    print("saved a checkpoint")

print('*'*20, 'SUCCESS','*'*20)

torch.Size([10, 200]) torch.Size([10, 200])
Variable containing:
 992.7575
[torch.FloatTensor of size 1]

Variable containing:
 501.2919
[torch.FloatTensor of size 1]



RuntimeError: inconsistent tensor size at /Users/soumith/miniconda2/conda-bld/pytorch_1493757319118/work/torch/lib/TH/generic/THTensorMath.c:193

In [ ]:
model[0].requires_grad

In [ ]:
for t, (qs, target) in enumerate(train_loader):

    q1 = Variable(qs[:,0,:])
    q2 = Variable(qs[:,1,:])
    target = Variable(target)

    pred1 = model(q1)
    pred2 = model(q2)
    loss = criterion(pred1, pred2, target)

    optimizer.zero_grad()
    loss.backward() # I'm currently not using the Hinge loss. 
    optimizer.step()

    print("trained one batch")



In [ ]:
q1 = Variable(qs[:,0,:])
embd.requires_grad = False
words = embd(q1)

In [ ]:
lengths = words.data.sum(2).abs().sign().sum(1)
s_ix = list(lengths.squeeze().cumsum(0).long() - lengths.squeeze().long())
e_ix = list(lengths.squeeze().cumsum(0).long())

# Filter out zero words 
mask = words.data.sum(2).abs().sign().expand_as(words).byte()
words = words.masked_select(Variable(mask)).view(-1, embd_dim)

In [ ]:
words = embd(q1)

In [65]:
for t, (qs, target) in enumerate(val_loader, 1):

    q0 = Variable(qs[:,0,:])
    pred0 = model(q0)

    scores = []
    losses = []

    for i in range(1, qs.size(1)):

        q = Variable(qs[:,i,:], volatile=True)
        pred = model(q)
        t = Variable(target[:,i], volatile=True)
        
        cos_dis = 1 - ((pred * pred0).sum(1) / (pred0.pow(2).sum(1).sqrt() * pred.pow(2).sum(1).sqrt())) 
        print(cos_dis)


        

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.

Variable containing:
 0
[torch.FloatTensor of size 1x1]

Variable containing:
 0
[torch.FloatTensor of size 1x1]



KeyboardInterrupt: 

In [63]:
pred0

Variable containing:

Columns 0 to 9 
 0.4882  0.5129  0.5122  0.4949  0.5074  0.5054  0.5056  0.5047  0.4903  0.4912

Columns 10 to 19 
 0.5070  0.4937  0.5015  0.5059  0.4896  0.5103  0.4969  0.4888  0.4794  0.5038

Columns 20 to 29 
 0.5042  0.4983  0.5047  0.5087  0.4833  0.5002  0.4814  0.5146  0.5083  0.4983

Columns 30 to 39 
 0.4991  0.4877  0.4987  0.4972  0.5035  0.4956  0.5050  0.4990  0.5062  0.5007

Columns 40 to 49 
 0.4966  0.5050  0.4919  0.4934  0.5043  0.4964  0.5063  0.5073  0.4997  0.4937

Columns 50 to 59 
 0.5061  0.4920  0.4966  0.5045  0.4998  0.5099  0.5015  0.4951  0.5025  0.5001

Columns 60 to 69 
 0.4978  0.4914  0.4910  0.5012  0.5042  0.4998  0.4898  0.5031  0.5067  0.4859

Columns 70 to 79 
 0.4869  0.4985  0.5151  0.4957  0.5011  0.5174  0.4847  0.4994  0.5014  0.4924

Columns 80 to 89 
 0.5035  0.4945  0.4930  0.5079  0.5094  0.5054  0.5017  0.4938  0.4886  0.5032

Columns 90 to 99 
 0.4926  0.4922  0.5127  0.5116  0.5058  0.4971  0.4889  0.5087  0.5132

In [66]:
q == q0

Variable containing:

Columns 0 to 12 
    0     0     0     0     0     0     0     0     0     0     1     1     1

Columns 13 to 25 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 26 to 37 
    1     1     1     1     1     1     1     1     1     1     1     1
[torch.ByteTensor of size 1x38]

In [49]:
(pred * pred0).sum(1)

Variable containing:
 50.0847
[torch.FloatTensor of size 1x1]

In [53]:
pred.pow(2).sum(1).sqrt() * pred0.pow(2).sum(1).sqrt()

Variable containing:
 50.0847
[torch.FloatTensor of size 1x1]

In [56]:
pred == pred0

Variable containing:

Columns 0 to 12 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 13 to 25 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 26 to 38 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 39 to 51 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 52 to 64 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 65 to 77 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 78 to 90 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 91 to 103 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 104 to 116 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Columns 117 to 129 
    1     1     1     1     1     1     1     1     1     1     1     1     1

Colu

In [55]:
pred0

Variable containing:

Columns 0 to 9 
 0.4882  0.5129  0.5122  0.4949  0.5074  0.5054  0.5056  0.5047  0.4903  0.4912

Columns 10 to 19 
 0.5070  0.4937  0.5015  0.5059  0.4896  0.5103  0.4969  0.4888  0.4794  0.5038

Columns 20 to 29 
 0.5042  0.4983  0.5047  0.5087  0.4833  0.5002  0.4814  0.5146  0.5083  0.4983

Columns 30 to 39 
 0.4991  0.4877  0.4987  0.4972  0.5035  0.4956  0.5050  0.4990  0.5062  0.5007

Columns 40 to 49 
 0.4966  0.5050  0.4919  0.4934  0.5043  0.4964  0.5063  0.5073  0.4997  0.4937

Columns 50 to 59 
 0.5061  0.4920  0.4966  0.5045  0.4998  0.5099  0.5015  0.4951  0.5025  0.5001

Columns 60 to 69 
 0.4978  0.4914  0.4910  0.5012  0.5042  0.4998  0.4898  0.5031  0.5067  0.4859

Columns 70 to 79 
 0.4869  0.4985  0.5151  0.4957  0.5011  0.5174  0.4847  0.4994  0.5014  0.4924

Columns 80 to 89 
 0.5035  0.4945  0.4930  0.5079  0.5094  0.5054  0.5017  0.4938  0.4886  0.5032

Columns 90 to 99 
 0.4926  0.4922  0.5127  0.5116  0.5058  0.4971  0.4889  0.5087  0.5132

In [32]:
for t, (qs, target) in enumerate(val_loader, 1):

    q0 = Variable(qs[:,0,:])
    pred0 = model(q0)

    scores = []
    losses = []

In [38]:
pred0

Variable containing:

Columns 0 to 9 
 1.0000  1.0000  1.0000  0.0000  0.0000  1.0000  0.0000  1.0000  0.0000  1.0000
 1.0000  1.0000  1.0000  0.0000  0.0000  1.0000  0.0000  1.0000  0.0000  1.0000
 0.0000  0.0000  0.0000  1.0000  1.0000  0.0000  1.0000  0.0000  1.0000  0.0000

Columns 10 to 19 
 1.0000  1.0000  0.0000  0.0000  0.0000  1.0000  0.0000  1.0000  0.0000  0.0000
 1.0000  1.0000  0.0000  0.0000  0.0000  1.0000  0.0000  1.0000  0.0000  0.0000
 0.0000  0.0000  1.0000  1.0000  1.0000  0.0000  1.0000  0.0000  1.0000  1.0000

Columns 20 to 29 
 1.0000  0.0000  1.0000  1.0000  0.0000  1.0000  1.0000  1.0000  0.0000  1.0000
 1.0000  0.0000  1.0000  1.0000  0.0000  1.0000  1.0000  1.0000  0.0000  1.0000
 0.0000  1.0000  0.0000  0.0000  1.0000  0.0000  0.0000  0.0000  1.0000  0.0000

Columns 30 to 39 
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  1.0000  1.0000  0.0000
 0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000  1.0000  1.0000  0.0000
 1.0000  1.0000  1.0000  

In [28]:
        
        
        
        print(pred.size(), pred0.size())
        print(pred0.pow(2).sum().sqrt() * pred.pow(2).sum().sqrt())
        print((pred * pred0).sum())
        cos_dis = 1 - ((pred * pred0).sum(1) / (pred0.pow(2).sum(1).sqrt() * pred.pow(2).sum(1).sqrt())) 
        scores.append(cos_dis.data[0])
        loss = criterion(pred0, pred, target)
        losses.append(loss.data[0])

torch.Size([10, 200]) torch.Size([10, 200])
Variable containing:
 992.7575
[torch.FloatTensor of size 1]

Variable containing:
 501.2919
[torch.FloatTensor of size 1]



RuntimeError: inconsistent tensor size at /Users/soumith/miniconda2/conda-bld/pytorch_1493757319118/work/torch/lib/TH/generic/THTensorMath.c:193

In [27]:
pred0

Variable containing:
 1.0000  1.0000  1.0000  ...   0.0000  0.0000  1.0000
 0.0000  0.0000  0.0000  ...   1.0000  1.0000  0.0000
 1.0000  1.0000  1.0000  ...   0.0000  0.0000  1.0000
          ...             ⋱             ...          
 1.0000  1.0000  1.0000  ...   0.0000  0.0000  1.0000
 1.0000  1.0000  1.0000  ...   0.0000  0.0000  1.0000
 1.0000  1.0000  1.0000  ...   0.0000  0.0000  1.0000
[torch.FloatTensor of size 10x200]